In [1]:
import pandas as pd
import numpy as np

In [163]:
votes = pd.read_csv("countypres_2000-2024.csv")
unemployment = pd.read_excel("LAUS_Annual_Unemployment_By_State_2000to2024.xlsx", skiprows=3)
med_income = pd.read_csv("Median_Household_Income_By_State_FRED.csv")
age = pd.read_csv("State_by_Age.csv")
education = pd.read_csv("State_Education_by_Degree.csv")
population = pd.read_csv("Population by Age and Sex - US, States, Counties.csv")

In [158]:
votes.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20250821,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20250821,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,OTHER,OTHER,113,17208,20250821,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,RALPH NADER,GREEN,160,17208,20250821,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20250821,TOTAL


In [161]:
votes["totalvotes"] = votes.groupby(["state", "year"])["candidatevotes"].transform("sum")
votes["dem"] = votes[votes["party"] == "DEMOCRAT"].groupby(["state", "year"])["candidatevotes"].transform("sum")
votes["rep"] = votes[votes["party"] == "REPUBLICAN"].groupby(["state", "year"])["candidatevotes"].transform("sum")
votes["other"] = votes[votes["party"] == "OTHER"].groupby(["state", "year"])["candidatevotes"].transform("sum")
votes["green"] = votes[votes["party"] == "GREEN"].groupby(["state", "year"])["candidatevotes"].transform("sum")
votes.head()

totalvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["totalvotes"],
    var_name = "party",
    value_name = "total_votes"
).drop_duplicates().dropna().drop(columns=["party"])

demvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["dem"],
    var_name = "party",
    value_name = "dem_votes"
).drop_duplicates().dropna().drop(columns=["party"])

repvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["rep"],
    var_name = "party",
    value_name = "rep_votes"
).drop_duplicates().dropna().drop(columns=["party"])

othervotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["other"],
    var_name = "party",
    value_name = "other_votes"
).drop_duplicates().dropna().drop(columns=["party"])

greenvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["green"],
    var_name = "party",
    value_name = "green_votes"
).drop_duplicates().dropna().drop(columns=["party"])

repvotes_melted.head()

,state,year,rep_votes
1,ALABAMA,2000,944409.0
269,ALASKA,2000,167398.0
433,ARIZONA,2000,781652.0
493,ARKANSAS,2000,472940.0
793,CALIFORNIA,2000,4567429.0


In [115]:
rename_states = {
    'LAUST010000000000003': 'Alabama',
    'LAUST020000000000003': 'Alaska',
    'LAUST040000000000003': 'Arizona',
    'LAUST050000000000003': 'Arkansas',
    'LAUST060000000000003': 'California',
    'LAUST080000000000003': 'Colorado',
    'LAUST090000000000003': 'Connecticut',
    'LAUST100000000000003': 'Delaware',
    'LAUST110000000000003': 'District of Columbia',
    'LAUST120000000000003': 'Florida',
    'LAUST130000000000003': 'Georgia',
    'LAUST150000000000003': 'Hawaii',
    'LAUST160000000000003': 'Idaho',
    'LAUST170000000000003': 'Illinois',
    'LAUST180000000000003': 'Indiana',
    'LAUST190000000000003': 'Iowa',
    'LAUST200000000000003': 'Kansas',
    'LAUST210000000000003': 'Kentucky',
    'LAUST220000000000003': 'Louisiana',
    'LAUST230000000000003': 'Maine',
    'LAUST240000000000003': 'Maryland',
    'LAUST250000000000003': 'Massachusetts',
    'LAUST260000000000003': 'Michigan',
    'LAUST270000000000003': 'Minnesota',
    'LAUST280000000000003': 'Mississippi',
    'LAUST290000000000003': 'Missouri',
    'LAUST300000000000003': 'Montana',
    'LAUST310000000000003': 'Nebraska',
    'LAUST320000000000003': 'Nevada',
    'LAUST330000000000003': 'New Hampshire',
    'LAUST340000000000003': 'New Jersey',
    'LAUST350000000000003': 'New Mexico',
    'LAUST360000000000003': 'New York',
    'LAUST370000000000003': 'North Carolina',
    'LAUST380000000000003': 'North Dakota',
    'LAUST390000000000003': 'Ohio',
    'LAUST400000000000003': 'Oklahoma',
    'LAUST410000000000003': 'Oregon',
    'LAUST420000000000003': 'Pennsylvania',
    'LAUST440000000000003': 'Rhode Island',
    'LAUST450000000000003': 'South Carolina',
    'LAUST460000000000003': 'South Dakota',
    'LAUST470000000000003': 'Tennessee',
    'LAUST480000000000003': 'Texas',
    'LAUST490000000000003': 'Utah',
    'LAUST500000000000003': 'Vermont',
    'LAUST510000000000003': 'Virginia',
    'LAUST530000000000003': 'Washington',
    'LAUST540000000000003': 'West Virginia',
    'LAUST550000000000003': 'Wisconsin',
    'LAUST560000000000003': 'Wyoming'
}

unemployment["Series ID"] = unemployment["Series ID"].replace(rename_states)

unemployment_renamed = unemployment.rename(columns = lambda x: x[-4:]).rename(columns={"s ID": "state"})

unemployment_melted = pd.melt(
    unemployment_renamed, id_vars = ["state"],
    value_vars = ["2000", "2004", "2008", "2012", "2016", "2020", "2024"],
    var_name = "year",
    value_name = "unemployment_rate"
)

election_years = ["2000", "2004", "2008", "2012", "2016", "2020", "2024"]
unemployment_melted = unemployment_melted[unemployment_melted["year"].isin(election_years)]
unemployment_melted["year"].unique()

array(['2000', '2004', '2008', '2012', '2016', '2020', '2024'],
      dtype=object)

In [117]:
abbr_to_state = {
    "ALA": "Alabama",
    "ARA": "Arkansas",
    "AZA": "Arizona",
    "CAA": "California",
    "COA": "Colorado",
    "CTA": "Connecticut",
    "DCA": "District of Columbia",
    "DEA": "Delaware",
    "FLA": "Florida",
    "GAA": "Georgia",
    "HIA": "Hawaii",
    "IDA": "Idaho",
    "ILA": "Illinois",
    "INA": "Indiana",
    "IOA": "Iowa",
    "KSA": "Kansas",
    "KYA": "Kentucky",
    "LAA": "Louisiana", 
    "MEA": "Maine",
    "MDA": "Maryland",
    "MAA": "Massachusetts",
    "MIA": "Michigan",
    "MNA": "Minnesota",
    "MSA": "Mississippi",
    "MOA": "Missouri",
    "MTA": "Montana",
    "NBA": "Nebraska",
    "NVA": "Nevada",
    "NHA": "New Hampshire",
    "NJA": "New Jersey",
    "NMA": "New Mexico",
    "NYA": "New York",
    "NCA": "North Carolina",
    "NDA": "North Dakota",
    "OHA": "Ohio",
    "OKA": "Oklahoma",
    "ORA": "Oregon",
    "PAA": "Pennsylvania",
    "RIA": "Rhode Island",
    "SCA": "South Carolina",
    "SDA": "South Dakota",
    "TNA": "Tennessee",
    "TXA": "Texas",
    "UTA": "Utah",
    "VAA": "Virginia",
    "VTA": "Vermont",
    "WAA": "Washington",
    "WIA": "Wisconsin",
    "WVA": "West Virginia",
    "WYA": "Wyoming"
}

med_income["observation_date"] = pd.to_datetime(med_income["observation_date"]).dt.year
med_income_renamed = med_income.rename(columns={"observation_date": "year"})

med_income_melted = pd.melt(
    med_income_renamed, id_vars = ["year"],
    value_vars = ['MEHOINUSALA672N', 'MEHOINUSARA672N', 'MEHOINUSAZA672N', 'MEHOINUSCAA672N', 'MEHOINUSCOA672N', 
                  'MEHOINUSCTA672N', 'MEHOINUSDCA672N', 'MEHOINUSDEA672N', 'MEHOINUSFLA672N', 'MEHOINUSGAA672N', 
                  'MEHOINUSHIA672N', 'MEHOINUSIAA672N', 'MEHOINUSIDA672N', 'MEHOINUSILA672N', 'MEHOINUSINA672N', 
                  'MEHOINUSKSA672N', 'MEHOINUSKYA672N', 'MEHOINUSLAA672N', 'MEHOINUSMAA672N', 'MEHOINUSMDA672N', 
                  'MEHOINUSMEA672N', 'MEHOINUSMIA672N', 'MEHOINUSMNA672N', 'MEHOINUSMOA672N', 'MEHOINUSMSA672N', 
                  'MEHOINUSMTA672N', 'MEHOINUSNCA672N', 'MEHOINUSNDA672N', 'MEHOINUSNEA672N', 'MEHOINUSNHA672N', 
                  'MEHOINUSNJA672N', 'MEHOINUSNMA672N', 'MEHOINUSNVA672N', 'MEHOINUSNYA672N', 'MEHOINUSOHA672N', 
                  'MEHOINUSOKA672N', 'MEHOINUSORA672N', 'MEHOINUSPAA672N', 'MEHOINUSSCA672N', 'MEHOINUSSDA672N', 
                  'MEHOINUSTNA672N', 'MEHOINUSTXA672N', 'MEHOINUSUTA672N', 'MEHOINUSVAA672N', 'MEHOINUSVTA672N', 
                  'MEHOINUSWAA672N', 'MEHOINUSWIA672N', 'MEHOINUSWVA672N', 'MEHOINUSWYA672N'],
    var_name = "state",
    value_name = "median_income"
)

med_income_melted["state"] = med_income_melted["state"].str[8:11].map(abbr_to_state)
med_income_melted["year"] = med_income_melted["year"].astype(str)

med_income_melted = med_income_melted[med_income_melted["year"].isin(election_years)]
med_income_melted["year"].unique()

med_income_melted.head()

,year,state,median_income
0,2000,Alabama,60570
1,2001,Alabama,58790
2,2002,Alabama,62100
3,2003,Alabama,60270
4,2004,Alabama,57810


In [107]:
fips_to_states = {
    1: "Alabama",
    2: "Alaska",
    4: "Arizona",
    5: "Arkansas",
    6: "California",
    8: "Colorado",
    9: "Connecticut",
    10: "Delaware",
    11: "District of Columbia",
    12: "Florida",
    13: "Georgia",
    15: "Hawaii",
    16: "Idaho",
    17: "Illinois",
    18: "Indiana",
    19: "Iowa",
    20: "Kansas",
    21: "Kentucky",
    22: "Louisiana",
    23: "Maine",
    24: "Maryland",
    25: "Massachusetts",
    26: "Michigan",
    27: "Minnesota",
    28: "Mississippi",
    29: "Missouri",
    30: "Montana",
    31: "Nebraska",
    32: "Nevada",
    33: "New Hampshire",
    34: "New Jersey",
    35: "New Mexico",
    36: "New York",
    37: "North Carolina",
    38: "North Dakota",
    39: "Ohio",
    40: "Oklahoma",
    41: "Oregon",
    42: "Pennsylvania",
    44: "Rhode Island",
    45: "South Carolina",
    46: "South Dakota",
    47: "Tennessee",
    48: "Texas",
    49: "Utah",
    50: "Vermont",
    51: "Virginia",
    53: "Washington",
    54: "West Virginia",
    55: "Wisconsin",
    56: "Wyoming"
}

age["STATEFIP"] = age["STATEFIP"].replace(fips_to_states)
age = age.rename(columns = {"STATEFIP": "state", "YEAR": "year"})

med_age = age.loc[:, ["state", "year", "med_age"]]

,state,year,voting_pop
0,Alabama,2000,1684421
1,Alabama,2004,1833058
2,Alabama,2008,1994524
3,Alabama,2012,942514
4,Alabama,2016,1044143


In [109]:
education["STATEFIP"] = education["STATEFIP"].replace(fips_to_states)
education = education.rename(columns = {"STATEFIP": "state", "YEAR": "year"})

education["bach_per"] = education["bach_pop"] / education["total_pop"]

edu_bach = education.loc[:, ["state", "year", "bach_per"]]

edu_bach.head()

,state,year,bach_per
0,Alabama,2000,0.239566
1,Alabama,2004,0.238953
2,Alabama,2008,0.233907
3,Alabama,2012,0.504814
4,Alabama,2016,0.492835


In [179]:
population_renamed = population.rename(columns = {"Total Population": "total_pop", "Year": "year", "Description": "state"})
population_renamed = population_renamed.loc[:, ["state", "year", "total_pop"]]
population_renamed = population_renamed.loc[population_renamed["state"] != "U.S."]
population_renamed["year"] = population_renamed["year"].astype(str)
population_renamed = population_renamed[population_renamed["year"].isin(election_years)]

population_renamed.head()

,state,year,total_pop
25,Alabama,2000,4452173
29,Alabama,2004,4530729
33,Alabama,2008,4718206
37,Alabama,2012,4816632
41,Alabama,2016,4866824


In [181]:
unemployment_melted["year"] = unemployment_melted["year"].astype(int)
unemployment_melted["state"] = unemployment_melted["state"].str.upper()
med_income_melted["year"] = med_income_melted["year"].astype(int)
med_income_melted["state"] = med_income_melted["state"].str.upper()
med_age["state"] = med_age["state"].str.upper()
total_pop["state"] = total_pop["state"].str.upper()
edu_bach["state"] = edu_bach["state"].str.upper()
population_renamed["state"] = population_renamed["state"].str.upper()
population_renamed["year"] = population_renamed["year"].astype(int)

merge1 = pd.merge(totalvotes_melted, demvotes_melted, on=["state", "year"], how="left")
merge2 = pd.merge(merge1, repvotes_melted, on=["state", "year"], how="left")
merge3 = pd.merge(merge2, othervotes_melted, on=["state", "year"], how="left")
merge4 = pd.merge(merge3, greenvotes_melted, on=["state", "year"], how="left")
merge5 = pd.merge(merge4, unemployment_melted, on=["state", "year"], how="left")
merge6 = pd.merge(merge5, med_income_melted, on=["state", "year"], how="left")
merge7 = pd.merge(merge6, med_age, on=["state", "year"], how="left")
merge8 = pd.merge(merge7, edu_bach, on=["state", "year"], how="left")
merged_data = pd.merge(merge8, population_renamed, on=["state", "year"], how="left")

merged_data["dem_votes_per"] = merged_data["dem_votes"] / merged_data["total_votes"]
merged_data["rep_votes_per"] = merged_data["rep_votes"] / merged_data["total_votes"]
merged_data["other_votes_per"] = merged_data["other_votes"] / merged_data["total_votes"]
merged_data["green_votes_per"] = merged_data["green_votes"] / merged_data["total_votes"]
merged_data["voter_participation"] = merged_data["total_votes"] / merged_data["total_pop"]

merged_data.head()

merged_data.to_csv("")

,state,year,total_votes,dem_votes,rep_votes,other_votes,green_votes,unemployment_rate,median_income,med_age,bach_per,total_pop,dem_votes_per,rep_votes_per,other_votes_per,green_votes_per,voter_participation
0,ALABAMA,2000,1672551,695602.0,944409.0,14191.0,18349.0,4.7,60570.0,43.0,0.239566,4452173,0.415893,0.564652,0.008485,0.010971,0.375671
1,ALASKA,2000,285530,79004.0,167398.0,10381.0,28747.0,6.3,NaN,41.0,0.284222,627963,0.276692,0.586271,0.036357,0.100679,0.454692
2,ARIZONA,2000,1534113,685341.0,781652.0,21475.0,45645.0,4.0,68020.0,44.0,0.268370,5160586,0.446734,0.509514,0.013998,0.029753,0.297275
3,ARKANSAS,2000,921781,422768.0,472940.0,12652.0,13421.0,4.2,50780.0,44.0,0.192445,2678588,0.458643,0.513072,0.013726,0.014560,0.344129
4,CALIFORNIA,2000,10965856,5861203.0,4567429.0,118517.0,418707.0,4.9,80050.0,42.0,0.328816,33987977,0.534496,0.416514,0.010808,0.038183,0.322639
